In [1]:
import pandas as pd
import csv
import psycopg2
import logging
import config


ModuleNotFoundError: No module named 'config'

In [ ]:
host="localhost"
database="ds"


def create_connection(db_file):    
    conn = None
    try:
        conn = psycopg2.connect(host=host,
                        database=database,
                        user=config.user,
                        password=config.password
                        )         
        print(psycopg2.__version__)
    except IOError  as e:
        print(e) 
    return conn

In [ ]:
def create_Resources_table(conn):
    
    cur = conn.cursor()    
    create_Resources_table = """
        CREATE TABLE IF NOT EXISTS Resources (
            Link        Text,
            Title       Text,
            Description Text  ) 
        
        """
    cur.execute(create_Resources_table)    
 

In [ ]:
def import_from_csv_to_table(conn):
    
    cur = conn.cursor()    
    data = None
    try:
        data =  open('clean_resource.csv', 'r')
        columns = data.readline().strip('\n').split(',')
        print("Data Imported Successfully.")
    except IOError:
        logging.exception('')        
    if not data:
        raise ValueError('No data available')

In [ ]:
def select_2_rows_resource(conn):
    
    cur = conn.cursor()
    
    query1 = """
        SELECT * FROM Resources limit 2         
        """
    cur.execute(query1) 
    rows = cur.fetchall()
 
    for row in rows:
        print(f"Link:\n\r{row[0]}\n\rTitle:\n\r{row[1]}\n\rDescription:\n\r {row[2]}")  

In [ ]:
def main():
    # create a database connection
    conn = create_connection(database)
    with conn: 
        create_Resources_table(conn)        
        
        import_from_csv_to_table(conn)        
        
        print("Select 2 rows of table:")
        select_2_rows_resource(conn)
        
    conn.commit()
    conn.close()    
 
if __name__ == '__main__':
    main()